# Validação Cruzada

### Essa Tecnica é utilizada para testar a peformace do modelo de machine learning

*Conceito Retirado do* ***Capitulo 5*** *apartir da* ***pagina 126*** *do livro* ***Data Science para Negocios***

Ele consiste em dividir o dataset em ***K*** partes.  
Cada parte é utilizada por vez como retenção enquanto a junção dos outros  
são usados como treinamento.  
**Treinamento:** (k - 1)/ k  
**Teste:** 1/k  
Por ultimo é gerado ***k*** resultado de desempenho(porcentagem de acerto em relação a lista de resposta esperada da *rentenção*)  
*rentenção: dados utilizados para teste* 

## Criação do modelo e puxar dataset

In [1]:
# importa as bibliotecas importantes
from sklearn import datasets, tree, svm, metrics
from sklearn.model_selection import cross_val_predict
import numpy as np

In [2]:
iris = datasets.load_iris()

In [3]:
# Gerando variaveis com dados da base e rusultado esperado 
base = iris['data']
resultado = iris['target']

In [4]:
# Função que Retorna o desempenho de acerto do modelo
def desempenho(res_teste, res_predit):
    valor = sum(res_teste == res_predit) / len(res_teste)
    return valor

#### Retenção Comum Com Arvore de Decisão

In [5]:
# gera Variavel com 20% do total da base
vinte_porcento = int(len(base)* 0.2)

In [6]:
# Gerado dados de Treinamento
base_modelo = base[:-vinte_porcento]
resultado_modelo = resultado[:-vinte_porcento]

In [7]:
# Gerado dados de Retenção(teste)
base_treinamento = base[-vinte_porcento:]
resultado_treinamento = resultado[-vinte_porcento:]

In [8]:
clf_t = tree.DecisionTreeClassifier().fit(base_modelo, resultado_modelo)

In [9]:
result_predicao_t = clf_t.predict(base_treinamento)

In [10]:
'Acerto de %.1f%%' %(desempenho(resultado_treinamento, result_predicao_t)*100)

'Acerto de 73.3%'

#### Retenção Comum Com SVM

In [11]:
clf_s = svm.SVC(gamma=0.001, C=100.).fit(base_modelo, resultado_modelo)

In [12]:
result_predicao_s = clf_s.predict(base_treinamento)

In [13]:
'Acerto de %.1f%%' %(desempenho(resultado_treinamento, result_predicao_s)*100)

'Acerto de 80.0%'

### Função que Calcula a Validação Cruzada

In [14]:
def dividir_partes(base, resultado, k):
    tam_parte = int(len(base)/k)
    count = 0
    base_partes = dict()
    result_partes = dict()
    
    for i in range(0, len(base), tam_parte):
        count += 1
        base_partes[count] = base[i:i+tam_parte]
        result_partes[count] = resultado[i:i+tam_parte]

    return base_partes, result_partes

In [15]:
def validacao_cruzada(base, resultado, classe_ml, k=5):
    r_base, r_result = dividir_partes(base, resultado, k)
    desempenho_lst = list()
    
    for key in list(r_base.keys()):
        abt = r_base[key]
        art = r_result[key]
        
        aux_base = list()
        aux_result = list()
        
        for i_key in list(r_base.keys()):
            if i_key == key:
                continue
            aux_base.append(r_base[i_key])
            aux_result.append(r_result[i_key])
            
        abm = np.concatenate(aux_base)
        arm = np.concatenate(aux_result)
    
        clf = classe_ml.fit(abm, arm)
        rp = clf.predict(abt)
        desempenho_lst.append(desempenho(art, rp))
    return np.array(desempenho_lst).mean(), np.array(desempenho_lst).std()

#### Validação Cruzada com Arvore de Decissão

In [16]:
valor = validacao_cruzada(base,resultado,tree.DecisionTreeClassifier())

In [17]:
str_result = 'Media: %.2f%%\nDesvio Padrão: %.3f' %(valor[0]*100, valor[1])
print(str_result)

Media: 89.33%
Desvio Padrão: 0.098


#### Validação Cruzada com SVM

In [18]:
valor_s = validacao_cruzada(base,resultado,svm.SVC(gamma=0.001, C=100.))

In [19]:
str_result = 'Media: %.2f%%\nDesvio Padrão: %.3f' %(valor_s[0]*100, valor_s[1])
print(str_result)

Media: 92.00%
Desvio Padrão: 0.078


### Usando Ferramentas da Biblioteca Sklearn

#### Validação Cruzada com Avore de Decisão

In [20]:
resultado_final_t = cross_val_predict(tree.DecisionTreeClassifier(), base, resultado, cv=5)

In [21]:
texto_t = 'Acuracia: %.2f%%' % (metrics.accuracy_score(resultado_final_t, resultado)*100)
print(texto_t)

Acuracia: 96.00%


#### Validação Cruzada com SVM

In [22]:
resultado_final_s = cross_val_predict(svm.SVC(gamma=0.001, C=100.), base, resultado, cv=5)

In [23]:
texto_s = 'Acuracia: %.2f%%' % (metrics.accuracy_score(resultado_final_s, resultado)*100)
print(texto_s)

Acuracia: 98.00%


***Obs:*** *O algoritmo do* ***sklearn*** *fuciona de forma diferente, onde ele tras um lista com todas as repostas juntas, enquanto o da função feito acima retorna a acuracia de cada parte e no final é feito a Media e Desvio padrão* ***como é proposto no livro.***